<a href="https://colab.research.google.com/github/MasakazuNaganuma/WhirlwindTourOfPython/blob/master/10-Iterators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<!--BOOK_INFORMATION-->
<img align="left" style="padding-right:10px;" src="https://github.com/MasakazuNaganuma/WhirlwindTourOfPython/blob/master/fig/cover-small.jpg?raw=1">
*This notebook contains an excerpt from the [Whirlwind Tour of Python](http://www.oreilly.com/programming/free/a-whirlwind-tour-of-python.csp) by Jake VanderPlas; the content is available [on GitHub](https://github.com/jakevdp/WhirlwindTourOfPython).*

*The text and code are released under the [CC0](https://github.com/jakevdp/WhirlwindTourOfPython/blob/master/LICENSE) license; see also the companion project, the [Python Data Science Handbook](https://github.com/jakevdp/PythonDataScienceHandbook).*


<!--NAVIGATION-->
< [Errors and Exceptions](09-Errors-and-Exceptions.ipynb) | [Contents](Index.ipynb) | [List Comprehensions](11-List-Comprehensions.ipynb) >

# イテレータ
# Iterators

Often an important piece of data analysis is repeating a similar calculation, over and over, in an automated fashion.
For example, you may have a table of a names that you'd like to split into first and last, or perhaps of dates that you'd like to convert to some standard format.
One of Python's answers to this is the *iterator* syntax.
We've seen this already with the ``range`` iterator:

データ分析の重要な部分は、自動化された方法で同じような計算を何度も何度も繰り返すことです。
例えば、名前の表を最初と最後に分割したい、あるいは日付の表を何らかの標準的な形式に変換したい、といったことがあるかもしれません。
これに対するPythonの答えの一つが*iterator*構文です。
これはすでに ``range`` のイテレータで見てきました。

In [0]:
for i in range(10):
    print(i, end=' ')

0 1 2 3 4 5 6 7 8 9 

Here we're going to dig a bit deeper.
It turns out that in Python 3, ``range`` is not a list, but is something called an *iterator*, and learning how it works is key to understanding a wide class of very useful Python functionality.

ここではもう少し深く掘り下げてみましょう。
Python 3では、``range``はリストではなく、*iterator*と呼ばれるものであり、それがどのように動作するのかを学ぶことは、Pythonの非常に便利な機能を理解するための鍵となります。

## リストの反復処理
## Iterating over lists

Iterators are perhaps most easily understood in the concrete case of iterating through a list.
Consider the following:

イテレータは、リストを反復するという具体的なケースで最も簡単に理解できるでしょう。
次のように考えてみてください。

In [0]:
for value in [2, 4, 6, 8, 10]:
    # do some operation
    print(value + 1, end=' ')

3 5 7 9 11 

The familiar "``for x in y``" syntax allows us to repeat some operation for each value in the list.
The fact that the syntax of the code is so close to its English description ("*for [each] value in [the] list*") is just one of the syntactic choices that makes Python such an intuitive language to learn and use.

But the face-value behavior is not what's *really* happening.
When you write something like "``for val in L``", the Python interpreter checks whether it has an *iterator* interface, which you can check yourself with the built-in ``iter`` function:

親しみのある"``for x in y``"構文では、リスト内の各値に対して何らかの操作を繰り返すことができます。
このコードの構文が英語の記述にとても近いという事実は、Pythonが直感的に学び、使うことができる言語であるための構文の選択の一つに過ぎません。

しかし、表面上の値の振る舞いは、実際に起こっていることではありません。
あなたが"``for val in L``"のようなことを書くとき、Pythonインタプリタはそれが*iterator*インターフェースを持っているかどうかをチェックします。

In [0]:
iter([2, 4, 6, 8, 10])

It is this iterator object that provides the functionality required by the ``for`` loop.
The ``iter`` object is a container that gives you access to the next object for as long as it's valid, which can be seen with the built-in function ``next``:

このイテレータオブジェクトが ``for`` ループで必要とされる機能を提供します。
``iter``オブジェクトは有効な限り次のオブジェクトにアクセスできるようにするためのコンテナで、組み込みの関数 ``next``で見ることができます。

In [0]:
I = iter([2, 4, 6, 8, 10])

In [0]:
print(next(I))

2


In [0]:
print(next(I))

4


In [0]:
print(next(I))

6


What is the purpose of this level of indirection?
Well, it turns out this is incredibly useful, because it allows Python to treat things as lists that are *not actually lists*.

このレベルのインダイレクトの目的は何でしょうか？
これは信じられないほど便利なことがわかりました。

## ``range()``: リストは常にリストとは限らない
## ``range()``: A List Is Not Always a List
Perhaps the most common example of this indirect iteration is the ``range()`` function in Python 3 (named ``xrange()`` in Python 2), which returns not a list, but a special ``range()`` object:

おそらくこの間接的な繰り返しの最も一般的な例は、Python 3の ``range()``関数(Python 2では ``xrange()``と名付けられています)で、これはリストではなく、特別な ``range()``オブジェクトを返します。

In [0]:
range(10)

range(0, 10)

``range``, like a list, exposes an iterator:

``range``はリストのようにイテレータを公開します。

In [0]:
iter(range(10))

So Python knows to treat it *as if* it's a list:

Pythonはリストであるかのように扱うことを知っています。

In [0]:
for i in range(10):
    print(i, end=' ')

0 1 2 3 4 5 6 7 8 9 

The benefit of the iterator indirection is that *the full list is never explicitly created!*
We can see this by doing a range calculation that would overwhelm our system memory if we actually instantiated it (note that in Python 2, ``range`` creates a list, so running the following will not lead to good things!):

イテレータのインダイレクトの利点は、*完全なリストが明示的に作成されることがないことです！*。
これは、実際にインスタンス化した場合にシステムメモリを圧倒するような範囲計算を行うことでわかります(Python 2では、``range``はリストを作成するので、以下のように実行しても良い結果にはならないことに注意してください！)。

In [0]:
N = 10 ** 12
for i in range(N):
    if i >= 10: break
    print(i, end=', ')

0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 

If ``range`` were to actually create that list of one trillion values, it would occupy tens of terabytes of machine memory: a waste, given the fact that we're ignoring all but the first 10 values!

In fact, there's no reason that iterators ever have to end at all!
Python's ``itertools`` library contains a ``count`` function that acts as an infinite range:

もし ``range``が実際に1兆個の値のリストを作成したとしたら、それは何十テラバイトものマシンメモリを占有することになります。

実際には、イテレータが終了する必要はありません。
Python の ``itertools`` ライブラリには、無限の範囲として振る舞う ``count`` 関数があります。

In [0]:
from itertools import count

for i in count():
    if i >= 10:
        break
    print(i, end=', ')

0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 

Had we not thrown-in a loop break here, it would go on happily counting until the process is manually interrupted or killed (using, for example, ``ctrl-C``).

ここでループブレークを入れなければ、プロセスが手動で中断されるか（例えば、``ctrl-C``などを使って）強制終了されるまで、喜んでカウントし続けるでしょう。

## 有用なイテレータ
## Useful Iterators

This iterator syntax is used nearly universally in Python built-in types as well as the more data science-specific objects we'll explore in later sections.
Here we'll cover some of the more useful iterators in the Python language:

このイテレータ構文はPythonの組み込み型や、後のセクションで紹介するデータサイエンスに特化したオブジェクトと同様にほぼ普遍的に使用されています。
ここでは、Python言語でより有用なイテレータのいくつかを取り上げます。

### ``enumerate``
Often you need to iterate not only the values in an array, but also keep track of the index.
You might be tempted to do things this way:

### ``enumerate``
多くの場合、配列の値だけでなく、インデックスを追跡する必要があります。
このようにしたいと思うかもしれません。

In [0]:
L = [2, 4, 6, 8, 10]
for i in range(len(L)):
    print(i, L[i])

0 2
1 4
2 6
3 8
4 10


Although this does work, Python provides a cleaner syntax using the ``enumerate`` iterator:

これは機能しますが、Pythonでは ``enumerate`` のイテレータを使ったよりクリーンな構文を提供しています。

In [0]:
for i, val in enumerate(L):
    print(i, val)

0 2
1 4
2 6
3 8
4 10


This is the more "Pythonic" way to enumerate the indices and values in a list.

これは、リスト内のインデックスと値を列挙する、より「Python的」な方法です。

### ``zip``
Other times, you may have multiple lists that you want to iterate over simultaneously.
You could certainly iterate over the index as in the non-Pythonic example we looked at previously, but it is better to use the ``zip`` iterator, which zips together iterables:

他にも、複数のリストを同時に反復処理したい場合もあるでしょう。
前に見たPythonicではない例のようにインデックスを反復処理することもできますが、それよりも ``zip`` のイテレータを使った方が良いでしょう。

In [0]:
L = [2, 4, 6, 8, 10]
R = [3, 6, 9, 12, 15]
for lval, rval in zip(L, R):
    print(lval, rval)

2 3
4 6
6 9
8 12
10 15


Any number of iterables can be zipped together, and if they are different lengths, the shortest will determine the length of the ``zip``.

イテレータブルは何個でも一緒にzipでまとめることができ、もしそれらの長さが異なる場合は、最も短いものが``zip``の長さを決定します。

### ``map`` and ``filter``
The ``map`` iterator takes a function and applies it to the values in an iterator:

``map``イテレータは関数を受け取り、それをイテレータ内の値に適用します。

In [0]:
# find the first 10 square numbers
square = lambda x: x ** 2
for val in map(square, range(10)):
    print(val, end=' ')

0 1 4 9 16 25 36 49 64 81 

The ``filter`` iterator looks similar, except it only passes-through values for which the filter function evaluates to True:

``filter``のイテレータは似ていますが、filter関数がTrueと評価される値だけを通過させることを除けば、似ています。

In [0]:
# find values up to 10 for which x % 2 is zero
is_even = lambda x: x % 2 == 0
for val in filter(is_even, range(10)):
    print(val, end=' ')

0 2 4 6 8 

The ``map`` and ``filter`` functions, along with the ``reduce`` function (which lives in Python's ``functools`` module) are fundamental components of the *functional programming* style, which, while not a dominant programming style in the Python world, has its outspoken proponents (see, for example, the [pytoolz](https://toolz.readthedocs.org/en/latest/) library).

``map``関数と``filter``関数、``reduce``関数（Pythonの``functools``モジュールに含まれています）は *関数型プログラミングスタイル* の基本的なコンポーネントです。

### 関数の引数としてのイテレータ
### Iterators as function arguments

We saw in [``*args`` and ``**kwargs``: Flexible Arguments](#*args-and-**kwargs:-Flexible-Arguments). that ``*args`` and ``**kwargs`` can be used to pass sequences and dictionaries to functions.
It turns out that the ``*args`` syntax works not just with sequences, but with any iterator:

以前、[ ``*args`` と ``**kwargs``: Flexible Arguments](#*args-and-**kwargs:-Flexible-Arguments)で見たように、 ``*args`` と ``**kwargs`` はシーケンスや辞書を関数に渡すために使うことができることがわかりました。
この構文はシーケンスだけでなく、任意のイテレータでも動作することがわかりました。

In [0]:
print(*range(10))

0 1 2 3 4 5 6 7 8 9


So, for example, we can get tricky and compress the ``map`` example from before into the following:

そこで、例えば、前の例の ``map`` を圧縮して以下のようにすることができます。

In [0]:
print(*map(lambda x: x ** 2, range(10)))

0 1 4 9 16 25 36 49 64 81


Using this trick lets us answer the age-old question that comes up in Python learners' forums: why is there no ``unzip()`` function which does the opposite of ``zip()``?
If you lock yourself in a dark closet and think about it for a while, you might realize that the opposite of ``zip()`` is... ``zip()``! The key is that ``zip()`` can zip-together any number of iterators or sequences. Observe:

このトリックを使うことで、Python学習者のフォーラムに出てくる古くからの疑問に答えることができます: なぜ、``zip()``の逆のことをする ``unzip()`` 関数がないのか?
暗いクローゼットの中に閉じこもってしばらく考えてみると、 ``zip()``の逆の機能があることに気づくかもしれません。``zip()``の反対は``...zip()``であることに気づくかもしれません。重要なのは、``zip()``は任意の数のイテレータやシーケンスをまとめて``zip()``にすることができるということです。重要なのは、``zip()``は任意の数のイテレータやシーケンスを zip 結合することができるということです。

In [0]:
L1 = (1, 2, 3, 4)
L2 = ('a', 'b', 'c', 'd')

In [0]:
z = zip(L1, L2)
print(*z)

(1, 'a') (2, 'b') (3, 'c') (4, 'd')


In [0]:
z = zip(L1, L2)
new_L1, new_L2 = zip(*z)
print(new_L1, new_L2)

(1, 2, 3, 4) ('a', 'b', 'c', 'd')


Ponder this for a while. If you understand why it works, you'll have come a long way in understanding Python iterators!

しばらく考えてみてください。なぜそれが動作するのかを理解すれば、Pythonのイテレータを理解する上で大きな道のりを歩むことができるでしょう!

## 特化したイテレータです。``itertools``
## Specialized Iterators: ``itertools``

We briefly looked at the infinite ``range`` iterator, ``itertools.count``.
The ``itertools`` module contains a whole host of useful iterators; it's well worth your while to explore the module to see what's available.
As an example, consider the ``itertools.permutations`` function, which iterates over all permutations of a sequence:


無限のイテレータ ``range`` の ``itertools.count`` を簡単に見てみました。
``itertools``モジュールにはたくさんの便利なイテレータが含まれています。
例として ``itertools.permutations`` 関数を考えてみましょう。

In [0]:
from itertools import permutations  #順列
p = permutations(range(3))
print(*p)

(0, 1, 2) (0, 2, 1) (1, 0, 2) (1, 2, 0) (2, 0, 1) (2, 1, 0)


Similarly, the ``itertools.combinations`` function iterates over all unique combinations of ``N`` values within a list:

同様に ``itertools.comombinations`` 関数はリストの中の ``N`` の値のユニークな組み合わせをすべて処理します。

In [0]:
from itertools import combinations  #組合せ
c = combinations(range(4), 2)
print(*c)

(0, 1) (0, 2) (0, 3) (1, 2) (1, 3) (2, 3)


Somewhat related is the ``product`` iterator, which iterates over all sets of pairs between two or more iterables:

少し関連しているのが ``product`` のイテレータで，2つ以上のイテレータブルの間のすべてのペアの集合をイテレートします．

In [0]:
from itertools import product
p = product('ab', range(3))
print(*p)

('a', 0) ('a', 1) ('a', 2) ('b', 0) ('b', 1) ('b', 2)


Many more useful iterators exist in ``itertools``: the full list can be found, along with some examples, in Python's [online documentation](https://docs.python.org/3.5/library/itertools.html).

``itertools``には他にも多くの便利なイテレータが存在します: Pythonの[オンラインドキュメント](https://docs.python.org/3.5/library/itertools.html)にその全リストがあります。

<!--NAVIGATION-->
< [Errors and Exceptions](09-Errors-and-Exceptions.ipynb) | [Contents](Index.ipynb) | [List Comprehensions](11-List-Comprehensions.ipynb) >